In [1]:
%pip install tensorflow_hub==0.16.1
%pip install tensorflow_text==2.15.0
%pip install tensorflow==2.15

INFO: pip is looking at multiple versions of tf-keras to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.0/404.0 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 6.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.4.1 requires six~=1.15.0, but you have six 1.16.0 which is incompatible.
tensorflow 2.4.1 requires typing-extensions~=3.7.4, but you have typing-extensions 4.9.0 which is incompatible.
tensorflow 2.4.1 requires wrapt~=1.12.1, but you have wrapt 1.14.1 which is incompatible.

[notice] A new release of pip is available: 22.0.3 ->

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
%pip install keras-core
from keras_nlp.models import BertPreprocessor, BertBackbone
import tensorflow_hub as hub
import tensorflow_text as text
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ImportError: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.8 from "/Users/munmun/opt/anaconda3/bin/python"
  * The NumPy version is: "1.20.1"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: dlopen(/Users/munmun/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so, 0x0002): tried: '/Users/munmun/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_multiarray_umath.cpython-38-darwin.so' (mach-o file, but is an incompatible architecture (have 'x86_64', need 'arm64')), '/System/Vol

In [6]:
df = pd.read_csv(r"/kaggle/input/person-recognition-dataset/data.csv")

In [7]:
df.head()

,Unnamed: 0,prompt,model
0,0,Counted the individuals present?,2
1,1,"Who's close by, ready to jump into the discuss...",0
2,2,Can you spot anyone you haven't met?,3
3,3,Any sightings of Eliana in sight?,1
4,4,Are there any unfamiliar faces in sight?,3


In [8]:
# I forgot to make x train, y train
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['prompt'],df['model'], stratify=df['model'])

# Because we're not doing binary classification
y_train = pd.get_dummies(y_train).astype(int).to_numpy()
y_test = pd.get_dummies(y_test).astype(int).to_numpy()

In [9]:
# Tokenize so we can train on a TPU
preprocessor = BertPreprocessor.from_preset("bert_base_en_uncased",trainable=True)
X_train_inputs = preprocessor(X_train)
X_test_inputs = preprocessor(X_test)

Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.txt' from model 'keras/bert/keras/bert_base_en_uncased/2' to your Kaggle notebook...


In [18]:
X_train_inputs

AttributeError: 'dict' object has no attribute 'shape'

In [15]:

# Use BERT
text_input = tf.keras.layers.Input(shape=(), dtype=tf.dict)
preprocessor = BertPreprocessor.from_preset("bert_small_en_uncased",trainable=True)
encoder_inputs = preprocessor(text_input)
encoder = BertBackbone.from_preset("bert_small_en_uncased")
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]      # [batch_size, 768].
sequence_output = outputs["sequence_output"]  # [batch_size, seq_length, 768].

# Now use that for classification
# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout-1")(pooled_output)
l = tf.keras.layers.Dense(64, activation='relu', name="dense-1")(l)
l = tf.keras.layers.Dropout(0.1, name="dropout-2")(l)
l = tf.keras.layers.Dense(16, activation='relu', name="dense-2")(l)
l = tf.keras.layers.Dropout(0.1, name="dropout-output")(l)
l = tf.keras.layers.Dense(4, activation='relu', name="output")(l)


# Create the model
# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

print(model.summary())
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=METRICS)

Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_small_en_uncased/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/bert/keras/bert_small_en_uncased/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.txt' from model 'keras/bert/keras/bert_small_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_small_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/bert/keras/bert_small_en_uncased/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/bert/keras/bert_small_en_uncased/2' to your Kaggle notebook...


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None)            │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_preprocessor_4 │ [(None, 512),     │          0 │ input_layer_3[0]… │
│ (BertPreprocessor)  │ (None, 512),      │            │                   │
│                     │ (None, 512)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bert_backbone       │ [(None, 512),     │ 28,763,648 │ bert_preprocesso… │
│ (BertBackbone)      │ (None, 512, 512)] │            │ bert_preprocesso… │
│                     │                   │            │ bert_preprocesso… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout-1 (Dropout) │ (None, 512)       │          0 │ bert_backbone[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense-1 (Dense)     │ (None, 64)        │     32,832 │ dropout-1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout-2 (Dropout) │ (None, 64)        │          0 │ dense-1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense-2 (Dense)     │ (None, 16)        │      1,040 │ dropout-2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout-output      │ (None, 16)        │          0 │ dense-2[0][0]     │
│ (Dropout)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 4)         │         68 │ dropout-output[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 28,797,588 (109.85 MB)

 Trainable params: 28,797,588 (109.85 MB)

 Non-trainable params: 0 (0.00 B)

None


In [16]:
y_train.shape

(628, 4)

In [17]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('my_temp_model.keras', monitor='val_accuracy', save_best_only=True, verbose=1)
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

history = model.fit(
    X_train, y_train, 
    validation_split=0.2, 
    epochs=25,
    callbacks=[checkpoint, earlystopping],
    batch_size=16
)

Epoch 1/25
18/32 ━━━━━━━━━━━━━━━━━━━━ 2:33 11s/step - accuracy: 0.5164 - loss: 8.1427 - precision: 0.2473 - recall: 0.4594

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

# Get training history
train_loss = history.history['loss']
val_loss = history.history['val_loss']
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

# Plot training and validation loss
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot training and validation accuracy
plt.plot(train_acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [13]:
# Evaluate on test data
model.evaluate(x=X_test, y=y_test)

7/7 [==============================] - 1s 178ms/step - loss: 1.4284 - accuracy: 0.7500 - precision: 0.0000e+00 - recall: 0.0000e+00


[1.4284303188323975, 0.75, 0.0, 0.0]

In [ ]:
model.save("final_model.h5")

In [21]:
y_pred = model.predict(X_test)


7/7 [==============================] - 1s 161ms/step


NameError: name 'y_preed' is not defined

In [31]:
print(np.argmax(y_pred,axis=1))
print(np.argmax(y_test, axis=1))
y_test

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[2 2 0 3 2 3 3 3 3 2 3 2 3 3 2 0 0 0 3 0 1 3 0 1 2 1 0 2 0 3 2 2 0 2 1 1 3
 2 1 3 0 2 2 1 2 3 1 2 0 2 3 0 3 1 2 0 3 0 3 3 2 2 0 0 2 2 0 2 0 1 0 1 3 2
 2 0 0 1 1 1 2 3 3 3 3 0 0 2 2 1 0 3 3 2 2 1 1 1 0 1 0 1 0 2 3 0 1 0 1 2 3
 3 2 1 0 0 3 0 3 1 0 3 2 1 0 2 1 1 1 1 1 2 1 3 2 1 3 1 3 1 1 1 3 0 3 2 2 2
 3 0 1 0 1 2 3 2 1 1 0 0 2 3 3 2 2 2 2 2 0 2 1 0 3 3 0 0 1 3 2 3 1 2 3 3 0
 1 0 2 2 0 3 2 1 1 1 0 0 0 0 2 1 3 1 0 3 0 0 1 0 1]


array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 1, 0, 0],
       [0, 0,

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

y_prediction = np.argmax(y_pred, axis = 0)
y_test=np.argmax(y_test, axis=0)
#Create confusion matrix and normalizes it over predicted (columns)
result = confusion_matrix(y_test, y_prediction , normalize='pred')

# Plot confusion matrix as heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues', 
            xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


TypeError: Singleton array 10 cannot be considered a valid collection.